In [10]:
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import missingno as msno


Task Specification:
Patterns of behavior in different regions of the sensor view as defined by behavior zones.
1. Pedestrians using the ramp vs stairs
2. Bicycles using the sidewalk vs bikelane.

In each case, we want to see how counts vary over the course of the day.


So, some important things are object detection + DEEPSORT (object tracking) for the actual counting process.

In [11]:
behavior_zones = pd.read_csv('../../data/behavior_zones.csv')
data = pd.read_csv('../../data/data.csv')
img_filepath = ('../../data/sample_image.jpg')

In [35]:
# behavior_zones seems to contain regions in the sensor view. 
# Visualize to make sense of the regions:

img = Image.open(img_filepath)
Drawer = ImageDraw.Draw(img)
for _, rows in behavior_zones.iterrows():
    box=[rows['xmin'], rows['ymin'], rows['xmax'], rows['ymax']]
    Drawer.rectangle(box, fill=None)

img.show()

In [21]:
# Labels
behavior_zones

,behavior_zone,xmin,ymin,xmax,ymax
0,stairs,380,80,530,200
1,ramp,70,100,350,200
2,sidewalk,180,200,415,280
3,bikelane,180,290,415,380


In [43]:
# We have 1918 unique trackids.
data.describe()

,trackid,date,time,class,box,bottom_center,geo_bottom_center
count,29904,29904,29904,29904,29904,29904,29904
unique,1918,1,19153,3,27236,17212,17212
top,d89224a1727846f2a2d5721387a634a2,2020-05-03,13:20:46.551,pedestrian,"[489, 100, 512, 142]","[443, 130]","[40.69936122778756, -73.96268410079243]"
freq,404,29904,8,25493,25,57,57


In [44]:
# we only care about the pedestrians for Q1, and bicycles for Q2. This can be identified by 'class'
data['class'].value_counts()

pedestrian    25493
bicycle        4406
truck             5
Name: class, dtype: int64

Gameplan:
Create a class object of `pedestrian` and `bicycles` depending on 'class'. -- referring to DEEPSORT code.
Step 1. Verify if the `bottom_center` position is within any of the pre-set boundaries listed in `behavior_zone`. 
Step 2. If the point is inside the set boundary, update the counter. To ensure that we can take care of instances where a pedestrian reaches multiple locations, we should ensure that the pedestrian can qualify for multiple tracks.
Step 3. Test how many objects we find -- and see how far it is from 1918. Evaluate.

Step 4. Consider ways that I can repeat this. 